In [1]:
# intalling folium
!pip install folium

In [2]:
# import pandas, numpy, matplotlib, seaborn, folium, requests, json, BeautifulSoup

# library for data analysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle data in a vectorized manner
import numpy as np


# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
import folium 
import requests
import json
from bs4 import BeautifulSoup

# Nominatim converts an address into latitude and langitude values
from geopy.geocoders import Nominatim

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("all libraries have been imported");

all libraries have been imported


In [24]:
sgtc_df=pd.read_csv('Singapore Planning Area.csv')
sgtc_df.head()

,Planning Area,Area (km2),Population,Density (/km2),Secondary Education,Density of Student Population,Density of Student Population(/km2),Latitude,Longitude
0,Ang Mo Kio,13.94,163950,13400,10500.0,753.228121,753.228121,1.370073,103.849516
1,Bedok,21.69,279380,13000,19200.0,885.200553,885.200553,1.323976,103.930216
2,Bishan,7.62,88010,12000,5600.0,734.908136,734.908136,1.349057,103.749591
3,Bukit Batok,11.13,153740,14000,10400.0,934.411500,934.411500,1.349057,103.749591
4,Bukit Merah,14.34,151980,11000,7600.0,529.986053,529.986053,1.270439,103.828318


**map of Singapore**

In [17]:
# use Nominatim to find out the latitude and longitude of Toronto

address = 'Singapore, SG'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Singapore are {}. {}.'.format(latitude, longitude))

The geographical coordinate of Singapore are 1.357107. 103.8194992.


In [25]:
# store the Toronto's latitude and longitude info
singapore_latlong = [latitude, longitude]
print(singapore_latlong)

# use folium library to create a map of Toronto using latitude
# and longitude values
map_singapore = folium.Map(location=singapore_latlong, tiles="OpenStreetMap", zoom_start=10.5
                    )

# add markers to map
for lat, lng, label in zip(sgtc_df['Latitude'], sgtc_df['Longitude'],sgtc_df['Planning Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2.5,
        popup=label,
        color='#2E7D32',
        fill= True,
        fill_color='#2E7D32',
        fill_opacity=0.5,
        parse_html=False).add_to(map_singapore)
    
    
map_singapore

[1.357107, 103.8194992]


#### 2) Explore the first neighborhood in our dataframe
Deifine Foursquare Credentials and Version

In [26]:
plarea = sgtc_df.loc[0,'Planning Area']

plarea

'Ang Mo Kio'

Now let's explore the top 100 venues that are in Ang Mo kio within a radius of 500 meters.

In [27]:
CLIENT_ID = 'YRKZ44SHQRO4UDMIPWFL2WICIH5BY0KLFEEBPUWKKYD2NUUK'
CLIENT_SECRET = 'OJCVO344VVYQTUOLCTCUYQ12D2XUHI0TYFIOFBYSKSNRV5F0'
VERSION = '20180605'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: YRKZ44SHQRO4UDMIPWFL2WICIH5BY0KLFEEBPUWKKYD2NUUK
CLIENT_SECRET:OJCVO344VVYQTUOLCTCUYQ12D2XUHI0TYFIOFBYSKSNRV5F0


In [28]:

plarea_latitude = sgtc_df.loc[0, 'Latitude']
plarea_longitude = sgtc_df.loc[0, 'Longitude']

plarea_name = sgtc_df.loc[0, 'Planning Area']

print('Latitude and longitude values of {} are {}, {}.'.format(plarea_name,plarea_latitude,plarea_longitude))

Latitude and longitude values of Ang Mo Kio are 1.3700733, 103.8495157.


In [56]:
# set the number of venues of returned by Foursquare API to be 100
LIMIT = 200 

# define radius as 1000 meters
radius = 5000

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    plarea_latitude,
    plarea_longitude,
    radius,
    LIMIT)

# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YRKZ44SHQRO4UDMIPWFL2WICIH5BY0KLFEEBPUWKKYD2NUUK&client_secret=OJCVO344VVYQTUOLCTCUYQ12D2XUHI0TYFIOFBYSKSNRV5F0&v=20180605&ll=1.3700733,103.8495157&radius=8000&limit=200'

In [57]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecd1c9d0a2972001b2ad7bd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Singapore',
  'headerFullLocation': 'Singapore',
  'headerLocationGranularity': 'city',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 1.442073372000072,
    'lng': 103.92140189082396},
   'sw': {'lat': 1.298073227999928, 'lng': 103.77762950917604}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9f4875f964a520e21a37e3',
       'name': 'Bishan - Ang Mo Kio Park',
       'location': {'address': 'Ang Mo Kio Ave 1',
        'lat': 1.3622192498801895,
        'lng': 103.8462495803833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 1.362219249

In [58]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bishan - Ang Mo Kio Park,Park,1.362219,103.846250
1,Aramsa ~ The Garden Spa,Spa,1.362292,103.847602
2,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,Indian Restaurant,1.355269,103.836719
3,Chocolat N' Spice,Bakery,1.351867,103.837558
4,Tomyum Mama,Thai Restaurant,1.353415,103.834441


In [60]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the planning areas in Toronto

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
singapore_venues = getNearbyVenues(names=sgtc_df['Planning Area'],
                                   latitudes=sgtc_df['Latitude'],
                                   longitudes=sgtc_df['Longitude']
                                  )

Ang Mo Kio
Bedok
Bishan
Bukit Batok
Bukit Merah
Bukit Panjang
Bukit Timah
Choa Chu Kang
Clementi
Geylang
Hougang
Jurong East
Jurong West
Kallang
Marine Parade
Novena
Outram
Pasir Ris
Punggol
Queenstown
Sembawang
Sengkang
Serangoon
Tampines
Tanglin
Toa Payoh
Woodlands
Yishun


In [69]:
print(singapore_venues.shape)
singapore_venues.head()

(2800, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ang Mo Kio,1.370073,103.849516,Bishan - Ang Mo Kio Park,1.362219,103.846250,Park
1,Ang Mo Kio,1.370073,103.849516,Aramsa ~ The Garden Spa,1.362292,103.847602,Spa
2,Ang Mo Kio,1.370073,103.849516,Sin Ming Roti Prata (Faisal & Aziz Curry Musli...,1.355269,103.836719,Indian Restaurant
3,Ang Mo Kio,1.370073,103.849516,Chocolat N' Spice,1.351867,103.837558,Bakery
4,Ang Mo Kio,1.370073,103.849516,Tomyum Mama,1.353415,103.834441,Thai Restaurant


In [70]:
singapore_venues['Venue Category'].value_counts()

Hotel                                       194
Park                                        162
Chinese Restaurant                          124
Coffee Shop                                 107
Café                                         69
Bakery                                       63
Shopping Mall                                60
Ice Cream Shop                               59
Supermarket                                  55
Japanese Restaurant                          55
Food Court                                   52
Garden                                       48
Indian Restaurant                            47
Trail                                        46
Italian Restaurant                           46
Thai Restaurant                              45
Performing Arts Venue                        40
Nature Preserve                              36
Clothing Store                               36
Asian Restaurant                             34
Exhibit                                 

In [73]:
singapore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ang Mo Kio,100,100,100,100,100,100
Bedok,100,100,100,100,100,100
Bishan,100,100,100,100,100,100
Bukit Batok,100,100,100,100,100,100
Bukit Merah,100,100,100,100,100,100
Bukit Panjang,100,100,100,100,100,100
Bukit Timah,100,100,100,100,100,100
Choa Chu Kang,100,100,100,100,100,100
Clementi,100,100,100,100,100,100


In [74]:
print('There are {} unique categories.'.format(len(singapore_venues['Venue Category'].unique())))

There are 178 unique categories.


In [75]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = singapore_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name="Neighborhood"
first_col = singapore_onehot.pop(col_name)
singapore_onehot.insert(0, col_name, first_col)

singapore_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Butcher,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,General Entertainment,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hockey Arena,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Malay Restaurant,Massage Studio,Military Base,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Nature Preserve,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,Ang Mo Kio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ang Mo Kio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ang Mo Kio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ang Mo Kio,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ang Mo Kio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [76]:
singapore_onehot.shape

(2800, 179)

Analyze each Neighborhood

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped

,Neighborhood,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bay,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Butcher,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Restaurant,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,General Entertainment,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,High School,Historic Site,History Museum,Hockey Arena,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Malay Restaurant,Massage Studio,Military Base,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Nature Preserve,Noodle House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,Ang Mo Kio,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00,0.02,0.03,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.05,0.00,0.02,0.01,0.02,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.12,0.00,0.00,0.01,0.04,0.02,0.01,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.07,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01
1,Bedok,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.03,0.00,0.04,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.10,0.00,0.02,0.00,0.01,0.09,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.03,0.01,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.02,0.02,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0

In [78]:
singapore_grouped.shape

(28, 179)

In [79]:
num_top_venues = 5

for hood in singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ang Mo Kio----
                venue  freq
0               Hotel  0.12
1                Park  0.07
2       Shopping Mall  0.05
3  Chinese Restaurant  0.05
4      Ice Cream Shop  0.04


----Bedok----
                venue  freq
0  Chinese Restaurant  0.10
1         Coffee Shop  0.09
2                Park  0.06
3              Bakery  0.04
4               Beach  0.04


----Bishan----
                venue  freq
0  Chinese Restaurant  0.05
1                Park  0.04
2          Food Court  0.04
3  Italian Restaurant  0.04
4              Bakery  0.04


----Bukit Batok----
                venue  freq
0  Chinese Restaurant  0.05
1                Park  0.04
2          Food Court  0.04
3  Italian Restaurant  0.04
4              Bakery  0.04


----Bukit Merah----
         venue  freq
0        Hotel  0.14
1         Park  0.05
2   Waterfront  0.04
3  Coffee Shop  0.03
4     Wine Bar  0.03


----Bukit Panjang----
                venue  freq
0                Park  0.06
1             Exhibit  0.0

Let's put that into a pandas dataframe

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ang Mo Kio,Hotel,Park,Shopping Mall,Chinese Restaurant,Ice Cream Shop,Supermarket,Bubble Tea Shop,Cocktail Bar,Nature Preserve,Japanese Restaurant
1,Bedok,Chinese Restaurant,Coffee Shop,Park,Beach,Bakery,Café,BBQ Joint,Hotel,Furniture / Home Store,Supermarket
2,Bishan,Chinese Restaurant,Park,Bakery,Nature Preserve,Food Court,Italian Restaurant,Supermarket,Garden,Café,Shopping Mall
3,Bukit Batok,Chinese Restaurant,Park,Bakery,Nature Preserve,Food Court,Italian Restaurant,Supermarket,Garden,Café,Shopping Mall
4,Bukit Merah,Hotel,Park,Waterfront,Theme Park Ride / Attraction,Coffee Shop,Wine Bar,Resort,Garden,Performing Arts Venue,Scenic Lookout


In [83]:

neighborhoods_venues_sorted.shape

(28, 11)

In [88]:
neighborhoods_venues_sorted.to_csv('Singapore_neighborhoods_venues_sorted.csv', index = False)